<a href="https://colab.research.google.com/github/SunSlick2/emailSort/blob/main/Email_Sorter_Python_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import win32com.client
import pandas as pd
import datetime
import openpyxl
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
from tkcalendar import Calendar
import threading
import logging
import time
import json
import pythoncom # Required for COM object threading issues

class EmailSorter:
    """
    A class to sort emails in Outlook based on rules defined in an Excel file
    and configuration from a JSON file.
    Supports live monitoring and bulk processing modes.
    """

    def __init__(self, config_path='config.json'):
        """
        Initializes the EmailSorter with configuration, sets up paths,
        loads data from Excel.
        """
        self.config_path = config_path
        self.config = None
        self.xls_path = None
        self.log_live_path = None
        self.log_bulk_path = None
        self.log_invalid_path = None

        # Outlook objects will no longer be class attributes initialized here.
        # They will be thread-local.
        self.tables = {}
        self.cliente_emails = set()
        self.dacs_notmine_emails = set()
        self.client_keywords = set()
        self.notmine_keywords = set()
        self.subject_only_keywords = set() # For general subject-only keywords
        self.subject_to_delete_keywords = set() # For keywords specifically to move to 'ToDelete'
        self.smtp_cache = {}
        self.new_smtp_entries = {}

        self.live_running = False
        self.invalid_logger = None
        self.live_logger = None
        self.bulk_logger = None

        try:
            # Load configuration first to get log paths
            self._load_config()
            self.setup_paths()
            self.setup_logging() # Setup logging after paths are available

            # Load data from Excel (this can be done in the main thread)
            self.load_data()

            self.invalid_logger.info("EmailSorter initialized successfully.")
        except Exception as e:
            # Log initialization errors to console and general invalid log
            error_message = f"Initialization error: {e}"
            print(error_message)
            if self.invalid_logger:
                self.invalid_logger.error(f"InitializationError||EmailSorter.__init__|{error_message}")
            messagebox.showerror("Initialization Error", error_message)
            raise # Re-raise to prevent further execution with an invalid state

    def _load_config(self):
        """
        Loads configuration from the JSON file specified during initialization.
        Ensures essential paths and sheet mappings are present.
        """
        try:
            print(f"Attempting to load config from: {os.path.abspath(self.config_path)}")
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)

            print(f"Keys found in sheet_map: {list(self.config['sheet_map'].keys())}")

            # Validate essential config fields including the folder names
            required_keys = ['xls_path', 'log_live_path', 'log_bulk_path', 'log_invalid_path',
                             'dacs_my_folder_name', 'dacs_folder_name', # These are global names, not tied to sheet_map
                             'sheet_map']
            for key in required_keys:
                if key not in self.config:
                    raise ValueError(f"Missing required configuration key: '{key}'")

            # Validate sheet_map structure for columns/column and new 'match_field'
            required_sheet_map_keys = [
                'ClienteMailAddresses', 'ClientKeywords',
                'DACSNotMineEmail', 'DACSNotMineKeyword', 'SubjectOnlyKeywords', # Added new table
                'KeywordSubject_ToDelete', # Added new table
                'SMTPResolutionCache'
            ]
            for sheet_key in required_sheet_map_keys:
                if sheet_key not in self.config['sheet_map']:
                    raise ValueError(f"Missing required sheet map configuration for: '{sheet_key}'")

                sheet_config = self.config['sheet_map'][sheet_key]
                if 'sheet' not in sheet_config:
                    raise ValueError(f"Missing 'sheet' key for '{sheet_key}' in sheet_map.")

                # Specific checks for 'column' or 'columns'
                if sheet_key in ['ClienteMailAddresses', 'DACSNotMineEmail']:
                    if 'column' not in sheet_config:
                        raise ValueError(f"Missing 'column' key for '{sheet_key}' in sheet_map.")
                elif sheet_key in ['ClientKeywords', 'DACSNotMineKeyword', 'SubjectOnlyKeywords', 'KeywordSubject_ToDelete']: # Check match_field for keyword tables
                    if 'columns' not in sheet_config or not isinstance(sheet_config['columns'], list) or not sheet_config['columns']:
                        raise ValueError(f"Missing or invalid 'columns' key for '{sheet_key}' in sheet_map. Must be a non-empty list.")
                    if 'match_field' not in sheet_config or sheet_config['match_field'] not in ['subject_only', 'subject_and_body']:
                        raise ValueError(f"Missing or invalid 'match_field' key for '{sheet_key}' in sheet_map. Must be 'subject_only' or 'subject_and_body'.")

            # Validate the new 'ToDelete' folder name is also present if this is a destination
            if 'KeywordSubject_ToDelete' in self.config['sheet_map']:
                to_delete_folder_name = self.config['sheet_map']['KeywordSubject_ToDelete'].get('destination_name')
                if not to_delete_folder_name or not isinstance(to_delete_folder_name, str) or not to_delete_folder_name.strip():
                     raise ValueError(f"Missing or invalid 'destination_name' for 'KeywordSubject_ToDelete' in sheet_map.")


            print(f"Configuration loaded successfully from {self.config_path}")
        except FileNotFoundError:
            raise FileNotFoundError(f"Configuration file {self.config_path} not found.")
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON in configuration file: {e}")
        except ValueError as e:
            raise e # Re-raise validation errors

    def setup_paths(self):
        """Sets up file paths from the loaded configuration."""
        self.xls_path = self.config['xls_path']
        self.log_live_path = self.config['log_live_path']
        self.log_bulk_path = self.config['log_bulk_path']
        self.log_invalid_path = self.config['log_invalid_path']

        # Ensure log directories exist
        os.makedirs(os.path.dirname(self.log_live_path) or '.', exist_ok=True)
        os.makedirs(os.path.dirname(self.log_bulk_path) or '.', exist_ok=True)
        os.makedirs(os.path.dirname(self.log_invalid_path) or '.', exist_ok=True)

        print(f"Paths set: Excel='{self.xls_path}', LiveLog='{self.log_live_path}', BulkLog='{self.log_bulk_path}', InvalidLog='{self.log_invalid_path}'")

    def setup_logging(self):
        """Configures logging for live, bulk, and invalid email entries."""
        # Setup the invalid logger first, as other setups might log to it
        self.invalid_logger = self._create_logger('invalid_log', self.log_invalid_path, level=logging.ERROR)
        self.live_logger = self._create_logger('live_log', self.log_live_path, level=logging.INFO)
        self.bulk_logger = self._create_logger('bulk_log', self.log_bulk_path, level=logging.INFO)
        print("Logging setup complete.")

    def _create_logger(self, name, log_path, level=logging.INFO):
        """Helper to create and configure a logger."""
        logger = logging.getLogger(name)
        logger.setLevel(level)

        # Clear existing handlers to prevent duplicate logs if called multiple times
        if logger.handlers:
            for handler in list(logger.handlers):
                logger.removeHandler(handler)

        handler = logging.FileHandler(log_path, mode='a', encoding='utf-8')
        formatter = logging.Formatter('%(asctime)s|%(levelname)s|%(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

    def load_data(self):
        """
        Loads all necessary data (email addresses, keywords, SMTP cache)
        from the configured Excel file.
        """
        try:
            self.tables = pd.read_excel(self.xls_path, sheet_name=None, engine='openpyxl')
            print(f"Excel file '{self.xls_path}' loaded successfully.")
        except FileNotFoundError:
            error_msg = f"Excel file not found: {self.xls_path}"
            self.invalid_logger.error(f"FileLoadError||load_data|{error_msg}")
            raise FileNotFoundError(error_msg)
        except Exception as e:
            error_msg = f"Error reading Excel file: {e}"
            self.invalid_logger.error(f"ExcelReadError||load_data|{error_msg}")
            raise ValueError(error_msg)

        # Load email addresses
        try:
            self.cliente_emails = self._load_email_addresses('ClienteMailAddresses')
            print(f"Loaded {len(self.cliente_emails)} client email addresses.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|ClienteMailAddresses|load_data|{e}")
            print(f"Warning: Could not load ClienteMailAddresses: {e}")

        try:
            self.dacs_notmine_emails = self._load_email_addresses('DACSNotMineEmail')
            print(f"Loaded {len(self.dacs_notmine_emails)} non-mine email addresses.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|DACSNotMineEmail|load_data|{e}")
            print(f"Warning: Could not load DACSNotMineEmail: {e}")

        # Load keywords from multiple-column tables
        try:
            self.client_keywords = self._load_keywords('ClientKeywords')
            print(f"Loaded {len(self.client_keywords)} client keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|ClientKeywords|load_data|{e}")
            print(f"Warning: Could not load ClientKeywords: {e}")

        try:
            self.dacs_notmine_keywords = self._load_keywords('DACSNotMineKeyword')
            print(f"Loaded {len(self.dacs_notmine_keywords)} non-mine keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|DACSNotMineKeyword|load_data|{e}")
            print(f"Warning: Could not load DACSNotMineKeyword: {e}")

        try:
            self.subject_only_keywords = self._load_keywords('SubjectOnlyKeywords')
            print(f"Loaded {len(self.subject_only_keywords)} general subject-only keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|SubjectOnlyKeywords|load_data|{e}")
            print(f"Warning: Could not load SubjectOnlyKeywords: {e}")

        try:
            self.subject_to_delete_keywords = self._load_keywords('KeywordSubject_ToDelete')
            print(f"Loaded {len(self.subject_to_delete_keywords)} 'to delete' subject keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|KeywordSubject_ToDelete|load_data|{e}")
            print(f"Warning: Could not load KeywordSubject_ToDelete: {e}")

        # Load SMTP resolution cache
        try:
            self.smtp_cache = self._load_smtp_cache()
            print(f"Loaded {len(self.smtp_cache)} SMTP cache entries.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|SMTPResolutionCache|load_data|{e}")
            print(f"Warning: Could not load SMTPResolutionCache: {e}")

        self.new_smtp_entries = {} # Reset new entries on load

    def _load_email_addresses(self, table_key):
        """
        Loads email addresses from a specified Excel sheet and column.
        Expects a single 'column' key in sheet_config.
        """
        sheet_config = self.config['sheet_map'][table_key]
        sheet_name = sheet_config['sheet']
        column_name = sheet_config['column']

        if sheet_name not in self.tables:
            raise ValueError(f"Sheet '{sheet_name}' not found in Excel file for '{table_key}'")

        df = self.tables[sheet_name]
        if column_name not in df.columns:
            raise ValueError(f"Column '{column_name}' not found in sheet '{sheet_name}' for '{table_key}'")

        # Convert to string, drop NaNs, and convert to lowercase for case-insensitive comparison
        return set(df[column_name].dropna().astype(str).str.lower())

    def _load_keywords(self, table_key):
        """
        Loads keywords from a specified Excel sheet and multiple columns.
        Expects a 'columns' list key in sheet_config.
        """
        sheet_config = self.config['sheet_map'][table_key]
        sheet_name = sheet_config['sheet']
        columns = sheet_config['columns']  # Expecting a list of columns

        if sheet_name not in self.tables:
            raise ValueError(f"Sheet '{sheet_name}' not found in Excel file for '{table_key}'")

        df = self.tables[sheet_name]

        # Verify all columns exist
        missing_columns = [col for col in columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns {missing_columns} not found in sheet '{sheet_name}' for '{table_key}'")

        keywords = set()
        for column in columns:
            # Extract non-null values from each column, convert to string, and lowercase
            column_values = df[column].dropna().astype(str).str.lower()
            keywords.update(column_values)

        # Remove any empty strings that might have resulted from conversion
        keywords.discard('')
        return keywords

    def _load_smtp_cache(self):
        """
        Loads SMTP resolution cache from the 'SMTPResolutionCache' sheet.
        Expects 'EntryName' and 'SMTPAddress' columns.
        """
        cache_sheet_name = self.config['sheet_map']['SMTPResolutionCache']['sheet']
        if cache_sheet_name in self.tables:
            cache_df = self.tables[cache_sheet_name]
            if 'EntryName' in cache_df.columns and 'SMTPAddress' in cache_df.columns:
                # Create a dictionary mapping lowercased EntryName to lowercased SMTPAddress
                # Handle potential NaN values by filling with empty strings
                return dict(zip(
                    cache_df['EntryName'].fillna('').astype(str).str.lower(),
                    cache_df['SMTPAddress'].fillna('').astype(str).str.lower()
                ))
            else:
                self.invalid_logger.warning(
                    f"Missing 'EntryName' or 'SMTPAddress' columns in '{cache_sheet_name}' sheet. "
                    "SMTP cache will not be loaded."
                )
        else:
            self.invalid_logger.warning(
                f"Sheet '{cache_sheet_name}' not found in Excel file. SMTP cache will not be loaded."
            )
        return {} # Return empty dict if sheet or columns are missing

    def get_smtp_address(self, outlook_namespace, entry):
        """
        Resolves the SMTP email address for an Outlook recipient or sender entry.
        Uses a cache to store resolved addresses to speed up subsequent lookups.
        If not in cache, attempts to resolve using MAPI properties or falls back to 'Address' field.
        This method now takes outlook_namespace as an argument.
        """
        if not entry:
            self.invalid_logger.warning("NullEntry||get_smtp_address|Received a None entry.")
            return None

        name = getattr(entry, 'Name', '') or ''
        address = getattr(entry, 'Address', '') or ''
        # Use name or address as a key, prioritizing name if available, and convert to lowercase
        name_key = (name.lower() if name else address.lower()) or ''

        if not name_key:
            self.invalid_logger.warning(f"EmptyNameKey|Name: '{name}', Address: '{address}'|get_smtp_address|No usable identifier for SMTP lookup.")
            return None

        # Check cache first (case-insensitive)
        cached = self.smtp_cache.get(name_key)
        if cached:
            return cached

        smtp = None
        # Try to get SMTP from MAPI property (PR_SMTP_ADDRESS)
        try:
            smtp = entry.PropertyAccessor.GetProperty("http://schemas.microsoft.com/mapi/proptag/0x39FE001E")
            if smtp:
                smtp = smtp.lower()
        except Exception:
            pass # MAPI property might not exist, fall through to address

        # Fallback to address field if MAPI property lookup failed
        if not smtp and address:
            smtp = address.lower()

        if smtp:
            # Store new valid SMTP entries in a temporary dict to save later
            # Also update the in-memory cache for immediate use within the current run
            self.new_smtp_entries[name_key] = smtp
            self.smtp_cache[name_key] = smtp
            return smtp
        else:
            self.invalid_logger.info(f"NoSMTPResolution|Name: '{name}', Address: '{address}'|get_smtp_address|Could not resolve SMTP address.")
            return None

    def extract_addresses(self, outlook_namespace, mail):
        """
        Extracts all relevant email addresses (recipients and sender) from a mail item.
        Handles cases where 'To' field might be missing by checking CC or From.
        This method now takes outlook_namespace as an argument.
        """
        recipients = set()
        to_addresses = set() # To keep track of 'To' addresses specifically

        # Process To, CC, BCC recipients
        has_to_recipient = False
        try:
            for rec in mail.Recipients:
                smtp = self.get_smtp_address(outlook_namespace, rec) # Pass namespace
                if smtp:
                    recipients.add(smtp)
                    if rec.Type == 1: # olTo
                        to_addresses.add(smtp)
                        has_to_recipient = True
        except Exception as e:
            self.invalid_logger.error(f"RecipientParseError|{mail.Subject or 'NoSubject'}|extract_addresses|{e}")

        # If no 'To' recipient, try to use CC or From for logging/categorization
        if not has_to_recipient and not to_addresses:
            self.invalid_logger.warning(f"MissingToField|Subject: '{mail.Subject or 'NoSubject'}'|extract_addresses|No 'To' recipient found. Checking CC/From.")

        # Extract sender
        sender = None
        try:
            sender = self.get_smtp_address(outlook_namespace, mail.Sender) # Pass namespace
            if sender:
                recipients.add(sender)
        except Exception as e:
            self.invalid_logger.error(f"SenderParseError|{mail.Subject or 'NoSubject'}|extract_addresses|{e}")

        # If after all attempts, no recipients are found (e.g., malformed email), log it.
        if not recipients:
            self.invalid_logger.error(f"NoAddressesFound|Subject: '{mail.Subject or 'NoSubject'}'|extract_addresses|No sender or recipient addresses extracted.")
        return recipients

    def keyword_match(self, mail, keywords, match_field="subject_and_body"):
        """
        Checks if any of the provided keywords exactly matches a word in the
        subject or body of the email based on match_field. Case-insensitive.
        match_field can be "subject_only" or "subject_and_body".
        """
        try:
            subject = (mail.Subject or "").lower()
            subject_words = set(subject.replace('.', ' ').replace(',', ' ').replace(':', ' ').replace(';', ' ').replace('!', ' ').replace('?', ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ').replace('{', ' ').replace('}', ' ').replace('-', ' ').replace('_', ' ').replace('/', ' ').replace('\\', ' ').strip().split())

            if match_field == "subject_only":
                target_words = subject_words
            elif match_field == "subject_and_body":
                body = (mail.Body or "").lower()
                body_words = set(body.replace('.', ' ').replace(',', ' ').replace(':', ' ').replace(';', ' ').replace('!', ' ').replace('?', ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ').replace('{', ' ').replace('}', ' ').replace('-', ' ').replace('_', ' ').replace('/', ' ').replace('\\', ' ').strip().split())
                target_words = subject_words.union(body_words)
            else:
                self.invalid_logger.error(f"InvalidMatchField|{mail.Subject or 'NoSubject'}|keyword_match|Unknown match_field: {match_field}. Defaulting to subject_and_body.")
                body = (mail.Body or "").lower()
                body_words = set(body.replace('.', ' ').replace(',', ' ').replace(':', ' ').replace(';', ' ').replace('!', ' ').replace('?', ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ').replace('{', ' ').replace('}', ' ').replace('-', ' ').replace('_', ' ').replace('/', ' ').replace('\\', ' ').strip().split())
                target_words = subject_words.union(body_words)

            for keyword in keywords:
                if keyword in target_words:
                    return keyword # Return the first matching keyword
            return None
        except Exception as e:
            self.invalid_logger.error(f"KeywordMatchError|{mail.Subject or 'NoSubject'}|keyword_match|{e}")
            return None

    def log_email(self, logger, outlook_namespace, mail, match_info, dest_folder_name):
        """
        Logs processed email information to the specified logger.
        Format: Date|Time|Sender|Subject|MatchInfo|DestinationFolder
        This method now takes outlook_namespace as an argument.
        """
        try:
            sent_on = mail.SentOn # This is a PyTime object
            date_str = sent_on.strftime("%Y-%m-%d")
            time_str = sent_on.strftime("%H:%M:%S")

            sender_smtp = self.get_smtp_address(outlook_namespace, mail.Sender) or "Unknown"

            subject = (mail.Subject or "NoSubject").replace('|', ' ').replace('\n', ' ').strip()

            log_entry = f"{date_str}|{time_str}|{sender_smtp}|{subject}|{match_info}|{dest_folder_name}"
            logger.info(log_entry)
        except Exception as e:
            self.invalid_logger.error(
                f"LogFormatError|Subject: '{getattr(mail, 'Subject', 'NoSubject') or 'NoSubject'}'|"
                f"log_email|Failed to format log entry: {e}"
            )

    def process_email(self, outlook_namespace, mail, logger, dacs_my_folder_obj, dacs_folder_obj, to_delete_folder_obj):
        """
        Processes a single email:
        1. Extracts addresses and checks against 'mine' and 'not mine' email lists.
        2. If no email match, checks against 'mine' and 'not mine' keyword lists.
        3. Moves the email to the appropriate folder ('DACS-My' or 'DACS' or 'ToDelete')
           and logs the action.
        Only the first matching rule is applied.
        This method now takes outlook_namespace and target folder *objects* as arguments.
        It uses the name property of the folder objects for logging.
        """
        try:
            recipients = self.extract_addresses(outlook_namespace, mail)

            # Rule 1: Email address in ClienteMailAddresses (highest priority)
            if any(addr in self.cliente_emails for addr in recipients):
                mail.Move(dacs_my_folder_obj)
                self.log_email(logger, outlook_namespace, mail, "Matched by ClienteMailAddresses", dacs_my_folder_obj.Name)
                return True

            # Rule 2: Email address in DACSNotMineEmail (second priority)
            if any(addr in self.dacs_notmine_emails for addr in recipients):
                mail.Move(dacs_folder_obj)
                self.log_email(logger, outlook_namespace, mail, "Matched by DACSNotMineEmail", dacs_folder_obj.Name)
                return True

            # Rule 3: Keyword in subject ONLY from KeywordSubject_ToDelete (new, high priority for deletion)
            to_delete_keyword = self.keyword_match(mail, self.subject_to_delete_keywords, match_field="subject_only")
            if to_delete_keyword:
                mail.Move(to_delete_folder_obj)
                self.log_email(logger, outlook_namespace, mail, f"Matched by KeywordSubject_ToDelete:'{to_delete_keyword}' (Subject Only)", to_delete_folder_obj.Name)
                return True

            # Rule 4: Keyword in subject ONLY from SubjectOnlyKeywords (general subject-only keywords)
            subject_only_keyword = self.keyword_match(mail, self.subject_only_keywords, match_field="subject_only")
            if subject_only_keyword:
                mail.Move(dacs_my_folder_obj) # Assuming it should go to DACS-My
                self.log_email(logger, outlook_namespace, mail, f"Matched by SubjectOnlyKeywords:'{subject_only_keyword}' (Subject Only)", dacs_my_folder_obj.Name)
                return True

            # Rule 5: Keyword in subject/body from ClientKeywords
            client_keyword = self.keyword_match(mail, self.client_keywords, match_field="subject_and_body")
            if client_keyword:
                mail.Move(dacs_my_folder_obj)
                self.log_email(logger, outlook_namespace, mail, f"Matched by ClientKeywords:'{client_keyword}'", dacs_my_folder_obj.Name)
                return True

            # Rule 6: Keyword in subject/body from DACSNotMineKeyword
            notmine_keyword = self.keyword_match(mail, self.dacs_notmine_keywords, match_field="subject_and_body")
            if notmine_keyword:
                mail.Move(dacs_folder_obj)
                self.log_email(logger, outlook_namespace, mail, f"Matched by DACSNotMineKeyword:'{notmine_keyword}'", dacs_folder_obj.Name)
                return True

            # If no rules matched, log to original folder name
            self.log_email(logger, outlook_namespace, mail, "No matching rules", mail.Parent.Name)
            return False

        except Exception as e:
            subject = getattr(mail, 'Subject', 'Unknown') or 'NoSubject'
            self.invalid_logger.error(f"EmailProcessingError|Subject: '{subject}'|process_email|{e}")
            print(f"Error processing email '{subject}': {e}")
            return False # Indicate that processing failed for this email

    def process_folder(self, outlook_namespace, folder_to_process, logger, date_filter, dacs_my_folder_obj, dacs_folder_obj, to_delete_folder_obj):
        """
        Processes all emails within a given Outlook folder.
        Filters emails from the specified date onwards.
        Emails are processed in reverse order to handle deletion/moving without affecting iteration.
        This method takes outlook_namespace and target folder *objects* as arguments.
        """
        processed_count = 0
        folder_name = getattr(folder_to_process, 'Name', 'Unknown')
        print(f"Starting processing for folder: {folder_name}")

        try:
            if not hasattr(folder_to_process, 'Items'):
                self.invalid_logger.error(f"InvalidFolder|{folder_name}|process_folder|Folder object has no 'Items' attribute.")
                return 0

            messages = folder_to_process.Items
            messages.Sort("[ReceivedTime]", False) # Sort by ReceivedTime descending

            if date_filter:
                date_filter_outlook_str = date_filter.strftime('%d %b %Y')
                # Filter from the start date onwards (removed end date)
                filter_string = f"[SentOn] >= '{date_filter_outlook_str} 00:00 AM'"
                print(f"Applying Outlook filter: {filter_string}")

                try:
                    filtered_messages = messages.Restrict(filter_string)
                    total_messages_to_process = filtered_messages.Count
                    print(f"Found {total_messages_to_process} messages in {folder_name} matching date filter {date_filter}.")
                except Exception as restrict_error:
                    self.invalid_logger.error(f"OutlookFilterError|{folder_name}|process_folder|Failed to apply filter '{filter_string}': {restrict_error}. Processing all messages and filtering manually.")
                    print(f"Warning: Failed to apply Outlook filter: {restrict_error}. Processing all messages and filtering manually.")
                    filtered_messages = messages # Fallback to all messages, then filter manually below
                    total_messages_to_process = messages.Count # Initial count
            else:
                filtered_messages = messages
                total_messages_to_process = messages.Count
                print(f"Processing all {total_messages_to_process} messages in {folder_name} (no date filter).")

            current_message_count = filtered_messages.Count

            for i in range(current_message_count, 0, -1):
                try:
                    mail = filtered_messages.Item(i)

                    # Manual date check for robustness if Outlook's Restrict failed or is imprecise
                    if date_filter:
                        mail_date = mail.SentOn.date()
                        # Only skip if the mail date is *before* the filter date
                        if mail_date < date_filter:
                            continue

                    # Pass thread-local Outlook objects to process_email
                    if self.process_email(outlook_namespace, mail, logger, dacs_my_folder_obj, dacs_folder_obj, to_delete_folder_obj):
                        processed_count += 1

                except Exception as msg_error:
                    subject = getattr(mail, 'Subject', 'Unknown') or 'NoSubject'
                    self.invalid_logger.error(f"MessageAccessError|Folder: '{folder_name}', Subject: '{subject}'|process_folder|{msg_error}")
                    print(f"Error accessing or processing message in {folder_name}: {msg_error}")
                    continue

        except Exception as e:
            self.invalid_logger.critical(f"FolderProcessingError|{folder_name}|process_folder|{e}")
            print(f"Critical error processing folder {folder_name}: {e}")
            return 0

        return processed_count

    def run_live(self):
        """
        Runs the email sorter in live mode, continuously monitoring and
        processing emails received/sent today.
        Initializes Outlook COM objects locally within this thread.
        """
        self.live_running = True
        print("Starting live mode...")
        self.live_logger.info("Live mode started.")

        outlook_app = None
        outlook_namespace = None
        inbox_folder = None
        sent_folder = None
        dacs_my_folder_obj = None
        dacs_folder_obj = None
        to_delete_folder_obj = None # New folder object

        try:
            pythoncom.CoInitialize()
            outlook_app = win32com.client.Dispatch("Outlook.Application")
            outlook_namespace = outlook_app.GetNamespace("MAPI")

            inbox_folder = outlook_namespace.GetDefaultFolder(6)
            sent_folder = outlook_namespace.GetDefaultFolder(5)

            # Get folder names from config
            dacs_my_folder_name = self.config['dacs_my_folder_name']
            dacs_folder_name = self.config['dacs_folder_name']
            to_delete_folder_name = self.config['sheet_map']['KeywordSubject_ToDelete']['destination_name'] # Get from sheet_map

            try:
                dacs_my_folder_obj = inbox_folder.Folders(dacs_my_folder_name)
            except Exception:
                dacs_my_folder_obj = inbox_folder.Folders.Add(dacs_my_folder_name)
                print(f"Created '{dacs_my_folder_name}' folder in Inbox for live thread.")

            try:
                dacs_folder_obj = inbox_folder.Folders(dacs_folder_name)
            except Exception:
                dacs_folder_obj = inbox_folder.Folders.Add(dacs_folder_name)
                print(f"Created '{dacs_folder_name}' folder in Inbox for live thread.")

            try:
                to_delete_folder_obj = inbox_folder.Folders(to_delete_folder_name)
            except Exception:
                to_delete_folder_obj = inbox_folder.Folders.Add(to_delete_folder_name)
                print(f"Created '{to_delete_folder_name}' folder in Inbox for live thread.")


            print("Outlook initialized for live mode thread.")

            while self.live_running:
                today = datetime.date.today()
                processed_inbox = 0
                processed_sent = 0

                try:
                    processed_inbox = self.process_folder(outlook_namespace, inbox_folder, self.live_logger, today, dacs_my_folder_obj, dacs_folder_obj, to_delete_folder_obj)
                    processed_sent = self.process_folder(outlook_namespace, sent_folder, self.live_logger, today, dacs_my_folder_obj, dacs_folder_obj, to_delete_folder_obj)
                except Exception as e:
                    self.invalid_logger.critical(f"LiveModeFatalError||run_live|A critical error occurred in live mode: {e}")
                    print(f"A critical error occurred in live mode: {e}. Stopping.")
                    self.live_running = False

                total_processed = processed_inbox + processed_sent
                if total_processed > 0:
                    print(f"Processed {total_processed} emails in live mode. Sleeping for 10 seconds...")
                else:
                    print("No new emails to process in live mode. Sleeping for 10 seconds...")

                for _ in range(10):
                    if not self.live_running:
                        print("Live mode stopped by user request during sleep.")
                        break
                    time.sleep(1)

        except Exception as e:
            self.invalid_logger.critical(f"LiveThreadSetupError||run_live|Failed to set up Outlook in live mode thread: {e}")
            print(f"Failed to set up Outlook in live mode thread: {e}. Live mode aborted.")
            self.live_running = False

        finally:
            self.live_logger.info("Live mode stopped.")
            print("Live mode gracefully stopped.")
            if outlook_app:
                # Explicitly release COM objects. Error handling in case an object is already released/invalidated.
                try: del dacs_my_folder_obj
                except: pass
                try: del dacs_folder_obj
                except: pass
                try: del to_delete_folder_obj # Delete new folder object
                except: pass
                try: del inbox_folder
                except: pass
                try: del sent_folder
                except: pass
                try: del outlook_namespace
                except: pass
                try: del outlook_app
                except: pass
            pythoncom.CoUninitialize()

    def stop_live(self):
        """Signals the live mode to stop its execution loop."""
        self.live_running = False
        print("Stopping live mode...")

    def run_bulk(self, date_selected):
        """
        Runs the email sorter in bulk mode for a specified date.
        Initializes Outlook COM objects locally within this thread.
        """
        print(f"Starting bulk processing for date: {date_selected}...")
        self.bulk_logger.info(f"Bulk mode started for date: {date_selected}.")

        outlook_app = None
        outlook_namespace = None
        inbox_folder = None
        sent_folder = None
        dacs_my_folder_obj = None
        dacs_folder_obj = None
        to_delete_folder_obj = None # New folder object

        processed_inbox = 0
        processed_sent = 0

        try:
            pythoncom.CoInitialize()
            outlook_app = win32com.client.Dispatch("Outlook.Application")
            outlook_namespace = outlook_app.GetNamespace("MAPI")

            inbox_folder = outlook_namespace.GetDefaultFolder(6)
            sent_folder = outlook_namespace.GetDefaultFolder(5)

            # Get folder names from config
            dacs_my_folder_name = self.config['dacs_my_folder_name']
            dacs_folder_name = self.config['dacs_folder_name']
            to_delete_folder_name = self.config['sheet_map']['KeywordSubject_ToDelete']['destination_name'] # Get from sheet_map

            try:
                dacs_my_folder_obj = inbox_folder.Folders(dacs_my_folder_name)
            except Exception:
                dacs_my_folder_obj = inbox_folder.Folders.Add(dacs_my_folder_name)
                print(f"Created '{dacs_my_folder_name}' folder in Inbox for bulk thread.")

            try:
                dacs_folder_obj = inbox_folder.Folders(dacs_folder_name)
            except Exception:
                dacs_folder_obj = inbox_folder.Folders.Add(dacs_folder_name)
                print(f"Created '{dacs_folder_name}' folder in Inbox for bulk thread.")

            try:
                to_delete_folder_obj = inbox_folder.Folders(to_delete_folder_name)
            except Exception:
                to_delete_folder_obj = inbox_folder.Folders.Add(to_delete_folder_name)
                print(f"Created '{to_delete_folder_name}' folder in Inbox for bulk thread.")

            print("Outlook initialized for bulk mode thread.")

            processed_inbox = self.process_folder(outlook_namespace, inbox_folder, self.bulk_logger, date_selected, dacs_my_folder_obj, dacs_folder_obj, to_delete_folder_obj)
            processed_sent = self.process_folder(outlook_namespace, sent_folder, self.bulk_logger, date_selected, dacs_my_folder_obj, dacs_folder_obj, to_delete_folder_obj)

        except Exception as e:
            self.invalid_logger.critical(f"BulkModeFatalError||run_bulk|A critical error occurred in bulk mode: {e}")
            print(f"A critical error occurred in bulk mode: {e}.")

        finally:
            total_processed = processed_inbox + processed_sent
            print(f"Bulk processing completed. Processed {total_processed} emails for {date_selected}.")
            self.bulk_logger.info(f"Bulk mode completed for date: {date_selected}. Processed {total_processed} emails.")

            messagebox.showinfo("Bulk Processing Complete",
                                f"Processed {total_processed} emails for {date_selected}\n"
                                f"Inbox: {processed_inbox} emails\n"
                                f"Sent Items: {processed_sent} emails")
            if outlook_app:
                try: del dacs_my_folder_obj
                except: pass
                try: del dacs_folder_obj
                except: pass
                try: del to_delete_folder_obj # Delete new folder object
                except: pass
                try: del inbox_folder
                except: pass
                try: del sent_folder
                except: pass
                try: del outlook_namespace
                except: pass
                try: del outlook_app
                except: pass
            pythoncom.CoUninitialize()

    def save_smtp_cache(self):
        """
        Saves newly resolved SMTP entries to the 'SMTPResolutionCache' sheet in the Excel file.
        Appends new entries without overwriting existing ones.
        """
        if not self.new_smtp_entries:
            print("No new SMTP entries to save.")
            return

        try:
            wb = openpyxl.load_workbook(self.xls_path, keep_vba=True)

            cache_sheet_name = self.config['sheet_map']['SMTPResolutionCache']['sheet']
            if cache_sheet_name not in wb.sheetnames:
                ws = wb.create_sheet(cache_sheet_name)
                ws.append(['EntryName', 'SMTPAddress'])
                print(f"Created new sheet '{cache_sheet_name}' for SMTP cache.")
            else:
                ws = wb[cache_sheet_name]
                if ws.max_row == 0 or ws.cell(row=1, column=1).value not in ['EntryName', 'entryname', 'EntryName']:
                    ws.insert_rows(1)
                    ws.cell(row=1, column=1, value='EntryName')
                    ws.cell(row=1, column=2, value='SMTPAddress')
                    print(f"Added headers to existing sheet '{cache_sheet_name}'.")

            existing_entries = set()
            for row in ws.iter_rows(min_row=2, values_only=True):
                if row and row[0]:
                    existing_entries.add(str(row[0]).lower())

            entries_added = 0
            for entry_name, smtp_address in self.new_smtp_entries.items():
                if entry_name not in existing_entries:
                    ws.append([entry_name, smtp_address])
                    existing_entries.add(entry_name)
                    entries_added += 1
                else:
                    print(f"Skipping existing SMTP cache entry: {entry_name}")

            if entries_added > 0:
                wb.save(self.xls_path)
                print(f"Saved {entries_added} new SMTP entries to '{cache_sheet_name}' in '{self.xls_path}'.")
            else:
                print("No *new* unique SMTP entries were added to the cache.")

            wb.close()
            self.new_smtp_entries.clear()
        except Exception as e:
            error_msg = f"Failed to update SMTPResolutionCache in '{self.xls_path}': {e}"
            self.invalid_logger.critical(f"CacheSaveError||save_smtp_cache|{error_msg}")
            print(error_msg)
            messagebox.showerror("Cache Save Error", error_msg)

    def start_gui(self):
        """
        Starts the main Tkinter GUI for the Email Sorter, allowing users to
        select between live and bulk processing modes.
        """
        root = tk.Tk()
        root.title("Email Sorter v2.0")
        root.geometry("350x280")
        root.resizable(False, False)
        root.attributes('-topmost', True)

        header_label = tk.Label(root, text="Email Sorter", font=("Arial", 16, "bold"), fg="#333333")
        header_label.pack(pady=15)

        info_label = tk.Label(root, text="Choose operation mode:", font=("Arial", 10), fg="#555555")
        info_label.pack(pady=5)

        def pick_bulk():
            """Handles bulk mode selection, prompting for a date."""
            cal_win = tk.Toplevel(root)
            cal_win.title("Select Date for Bulk Processing")
            cal_win.geometry("300x320")
            cal_win.resizable(False, False)
            cal_win.attributes('-topmost', True)
            cal_win.grab_set()

            cal_info_label = tk.Label(cal_win, text="Select date to process:", font=("Arial", 10))
            cal_info_label.pack(pady=10)

            cal = Calendar(cal_win, selectmode='day', date_pattern='yyyy-mm-dd',
                           background="blue", foreground="white",
                           headersbackground="blue", headersforeground="white",
                           selectbackground="green", selectforeground="white",
                           normalbackground="lightgray", weekendbackground="darkgray")
            cal.pack(pady=10)

            def submit_date():
                selected_date = cal.selection_get()
                cal_win.destroy()
                root.destroy()

                threading.Thread(target=lambda: self.run_bulk(selected_date), daemon=True).start()

            button_frame = tk.Frame(cal_win)
            button_frame.pack(pady=10)

            tk.Button(button_frame, text="Process", command=submit_date,
                      bg="#28a745", fg="white", width=12, height=1,
                      font=("Arial", 10, "bold"), relief=tk.RAISED).pack(side=tk.LEFT, padx=10)
            tk.Button(button_frame, text="Cancel", command=cal_win.destroy,
                      bg="#dc3545", fg="white", width=12, height=1,
                      font=("Arial", 10, "bold"), relief=tk.RAISED).pack(side=tk.LEFT, padx=10)

        def pick_live():
            """Handles live mode selection, starting continuous monitoring."""
            root.destroy()

            live_win = tk.Tk()
            live_win.title("Live Mode - Email Sorter")
            live_win.geometry("320x160")
            live_win.resizable(False, False)
            live_win.attributes('-topmost', True)

            status_label = tk.Label(live_win, text="Live monitoring active...",
                                     font=("Arial", 12, "bold"), fg="green")
            status_label.pack(pady=20)

            info_label = tk.Label(live_win, text="Emails are being processed automatically in the background.",
                                   font=("Arial", 9), fg="#666666")
            info_label.pack(pady=5)

            def stop_and_close():
                self.stop_live()
                self.save_smtp_cache()
                live_win.destroy()

            tk.Button(live_win, text="Stop Live Mode", command=stop_and_close,
                      bg="#dc3545", fg="white", width=18, height=1,
                      font=("Arial", 10, "bold"), relief=tk.RAISED).pack(pady=20)

            threading.Thread(target=self.run_live, daemon=True).start()

            live_win.protocol("WM_DELETE_WINDOW", stop_and_close)
            live_win.mainloop()

        button_frame = tk.Frame(root)
        button_frame.pack(pady=20)

        tk.Button(button_frame, text="Run Live Mode", command=pick_live,
                  bg="#007bff", fg="white", width=18, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(pady=8)
        tk.Button(button_frame, text="Run Bulk Mode", command=pick_bulk,
                  bg="#ffc107", fg="white", width=18, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(pady=8)

        footer_label = tk.Label(root, text="Live: Monitors today's emails | Bulk: Process specific date",
                                 font=("Arial", 8), fg="gray")
        footer_label.pack(side=tk.BOTTOM, pady=10)

        def on_closing():
            if messagebox.askyesno("Exit", "Do you want to save the SMTP cache before exiting?"):
                self.save_smtp_cache()
            root.destroy()

        root.protocol("WM_DELETE_WINDOW", on_closing)
        root.mainloop()

def main():
    """Main function to run the Email Sorter application."""
    sorter = None
    try:
        sorter = EmailSorter()
        sorter.start_gui()
    except Exception as e:
        print(f"Error starting Email Sorter application: {e}")
        if sorter and sorter.invalid_logger:
            sorter.invalid_logger.critical(f"AppStartupError||main|{e}")
    finally:
        if sorter:
            sorter.save_smtp_cache()
        pass

if __name__ == "__main__":
    main()